In [1]:
import sys
import pandas as pd
# import yfinance as yf
# import kplot as kp

In [5]:
# kline plot with pyecharts and akshare

# from pyecharts.charts import Kline
from pyecharts.charts import Kline, Bar, Grid
from pyecharts import options as opts
import akshare as ak
import pandas as pd

# from pyecharts.globals import CurrentConfig,NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from typing import List, Tuple, Dict, Any, Union, Optional

def get_aks_KlineData(symbol, period="daily", adjust="qfq", **kwargs):
    data=ak.stock_zh_a_hist(symbol, period, adjust, **kwargs)
    data['color']=data['涨跌幅'].apply(lambda x:True if x>=0 else False)
    data['MA5']=data['收盘'].rolling(5).mean()
    data['MA20']=data['收盘'].rolling(20).mean()
    data['MA80']=data['收盘'].rolling(80).mean()
    data['MA320']=data['收盘'].rolling(320).mean()
    return data

def trans_aksdat_to_echartdat(df_data: pd.DataFrame) -> Tuple:
    """ """
    outdict={}
    for col in df_data.columns:
        outdict[col]=df_data[col].to_list()
    outdict['kline_data'] = df_data[["开盘", "收盘", "最低", "最高"]].values.tolist()
    return outdict


def draw_kline(
    date, kline_data, yaxis_name="Price", title="Stock K-Line Chart", markline=None
) -> Kline:
    """ """
    kline = (
        Kline()
        .add_xaxis(date)
        .add_yaxis(
            yaxis_name,
            y_axis=kline_data,
            markline_opts=opts.MarkLineOpts(
                data=(
                    [
                        opts.MarkLineItem(type_="max", name="Max"),
                        opts.MarkLineItem(type_="min", name="Min"),
                    ]
                    if markline
                    else None
                ),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                is_scale=True,
            ),
            datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
    )
    return kline


def draw_bar(date, volumn_data,color_data, yaxis_name="Volume", title="Stock Volume Chart") -> Bar:
    bar = (
        Bar().add_xaxis(date).add_yaxis(yaxis_name, y_axis=volumn_data)
    ).set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
        yaxis_opts=opts.AxisOpts(type_="value", is_scale=True),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="vertical"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
    )

    return bar


class kchartDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def getData(self, symbol, period="daily", adjust="qfq", **kwargs):
        data=get_aks_KlineData(symbol, period, adjust, **kwargs)
        self[symbol] = data
        return self[symbol]

    def plot_aks(self, symbol, period="daily", adjust="qfq", **kwargs):
        if symbol not in self.keys():
            self.getData(symbol, period, adjust, **kwargs)
        data_dict = trans_aksdat_to_echartdat(
            self[symbol]
        )
        kline = draw_kline(
            data_dict['日期'], data_dict['kline_data'], symbol, title="stock " + symbol + " Kline Chart"
        )

        volumn_bar = draw_bar(
            data_dict['日期'],
            data_dict['成交量'],
            data_dict['color']
        )

        grid = (
            Grid(init_opts=opts.InitOpts(width="900px", height="600px"))
            .add(
                kline,
                grid_opts=opts.GridOpts(pos_bottom="30%"),
                is_control_axis_index=True,
            )
            .add(
                volumn_bar,
                grid_opts=opts.GridOpts(pos_top="70%"),
            )
        )
        return grid
    


In [3]:
kchart = kchartDict()

In [ ]:
kchart.plot_aks(symbol='000005',period='monthly').render_notebook()

### 选股
- 查询所有股票名与代码

1. 运营状况分析
- 所属行业
- 近10年营业额上升趋势
- 营业额组成分析，近5年主营业务占比百分数，是否有转型趋势
- 近10年利润上升趋势
- 近10年毛利率改善，或一直大于 ***?***
- 近5年资产负债率，与行业平均值对比
- 近5年市盈率，与行业平均值对比

2. 资金指标
- 昨日第一天涨停
- 昨日第一天成交量大于近10日平均值的***倍数***
- 近 ***?*** 日成交量呈上升趋势
- 近 ***?*** 日资金净流入

3. 技术指标
- 穿过均线
- 

4. 类型
- 次新股


In [ ]:
# functions for stock with its symbol code
#
import numpy as np

def get_stock_info(
    symbol,
):  # 获取单只股的信息
    info = ak.stock_individual_info_em(symbol)
    # info1=ak.stock_individual_basic_info_xq(symbol)
    return (info,)  # info1


def get_income(symbol, years):
    data = ak.stock_yjbb_em(date="20220331")  # 年报季报业绩报表
    data = ak.stock_yjyg_em(date="20190331")  # 业绩预告
    return


def cc():
    ak.stock_zcfz_em(date="20240331")  # 资产负债表
    return


def get_main_industry(symbol, years):
    data = ak.stock_zygc_em(symbol)
    return data


def get_profile(symbol, years):
    return


def get_profile_lv(symbol, years):  # 毛利率
    return


def get_(symbol, years):  # 资产负债率
    return


def get_(symbol, years):  # 市盈率
    return


def get_top_days(symbol) -> int:  # 涨停天数
    return


def get_(symbol, days=11, today="today"):  # 当日、10日成交量平均值
    return


def linear(yd: list, xd: list = None):
    xd = xd if len(yd) == len(xd) else [i for i in range(len(yd))]
    yavr=sum(yd)/len(yd)
    np.polynomial
    # 线性回归
    return


def get_newStock():
    data = ak.stock_new_a_spot_em()
    return data

In [30]:
cc=[0,1,2,3,4]
linear(cc,[3,4,5,0])

[0, 1, 2, 3, 4] [0, 1, 2, 3, 4]


In [ ]:
get_stock_info('000001')

(   item                value
 0  股票代码               000001
 1  股票简称                 平安银行
 2   总股本        19405918198.0
 3   流通股        19405571850.0
 4   总市值  218704698091.459991
 5  流通市值       218700794749.5
 6    行业                   银行
 7  上市时间             19910403,)

In [ ]:
ak.stock_zcfz_em(date="20240331")

ConnectionError: HTTPSConnectionPool(host='datacenter-web.eastmoney.com', port=443): Max retries exceeded with url: /api/data/v1/get?sortColumns=NOTICE_DATE%2CSECURITY_CODE&sortTypes=-1%2C-1&pageSize=500&pageNumber=1&reportName=RPT_DMSK_FN_BALANCE&columns=ALL&filter=%28SECURITY_TYPE_CODE+in+%28%22058001001%22%2C%22058001008%22%29%29%28TRADE_MARKET_CODE%21%3D%22069001017%22%29%0A++++++++%28REPORT_DATE%3D%272024-03-31%27%29 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D4BE9350A0>: Failed to resolve 'datacenter-web.eastmoney.com' ([Errno 11001] getaddrinfo failed)"))